# <center> 3) Preparación de los Datos <center>

## Selección de Datos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
# !(pip install chainladder)
import chainladder as cl

print("pandas: " + pd.__version__)
print("numpy: " + np.__version__)
print("chainladder: " + cl.__version__)

pandas: 1.4.2
numpy: 1.22.0
chainladder: 0.8.18


In [2]:
data=pd.read_csv('wkcomp_pos.csv')

In [6]:
data.columns
# data.columns[4:11]

Index(['GRCODE', 'GRNAME', 'AccidentYear', 'DevelopmentYear', 'DevelopmentLag',
       'IncurLoss_D', 'CumPaidLoss_D', 'BulkLoss_D', 'EarnedPremDIR_D',
       'EarnedPremCeded_D', 'EarnedPremNet_D', 'Single', 'PostedReserve97_D'],
      dtype='object')

In [7]:
triangle = cl.Triangle(
    data,
    origin="AccidentYear",
    development="DevelopmentYear",
    columns=data.columns[4:11],
    index=["GRNAME"],
    cumulative=True,
)
triangle

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 7, 19, 19)"
Index:,[GRNAME]
Columns:,"[DevelopmentLag, IncurLoss_D, CumPaidLoss_D, BulkLoss_D, EarnedPremDIR_D, EarnedPremCeded_D, EarnedPremNet_D]"


In [8]:
triangle.index.head()

,GRNAME
0,Agway Ins Co
1,Alaska Nat Ins Co
2,Alaska Timber Ins Exchange
3,Allstate Ins Co Grp
4,American Contractors Ins Grp


In [11]:
print("Estructura del Chainladder triangle:", type(triangle.values))
print("Suma de los valores de la tabla de datos:", np.nansum(triangle.values))

Estructura del Chainladder triangle: <class 'numpy.ndarray'>
Suma de los valores de la tabla de datos: 768244598.0


Triángulo de pérdidas

In [18]:
triangle[[x for x in data.columns if("Loss" in x)]]

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 3, 19, 19)"
Index:,[GRNAME]
Columns:,"[IncurLoss_D, CumPaidLoss_D, BulkLoss_D]"


Frecuencias de la parte exploratoria para GRNAME, la cual nos mostraba los nombres de las 132 empresas en la base de datos.

In [27]:
freq=pd.DataFrame(data['GRNAME'].value_counts())
freq.index

Index(['Allstate Ins Co Grp', 'Bancinsure Inc', 'Harco Natl Ins Co',
       'Transportation Cas Ins Co', 'Beacon Mut Ins Co', 'Mapfre Reins Corp',
       'National American Ins Co', 'Midwest Family Mut Ins Co',
       'Associated Industries Ins Co', 'Lumbermens Underwriting Alliance',
       ...
       'Catholic Relief Ins Co Of Amer', 'Care West Ins Co', 'FFVA Mut Ins Co',
       'MO Employers Mut Ins Co', 'Martingale Natl Ins Co',
       'Hyundai Marine & Fire Ins Co Ltd', 'Agway Ins Co', 'Mada Ins Exchange',
       'Lumber Ins Cos', 'Tower Ins Co Of NY'],
      dtype='object', length=132)

In [31]:
triangle.loc[freq.index[0]].iloc[-1]['CumPaidLoss_D']

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,"70,571","155,905","220,744","251,595","274,156","287,676","298,499","304,873","321,808","325,322",,,,,,,,,
1989,"66,547","136,447","179,142","211,343","231,430","244,750","254,557","270,059","273,873","277,574",,,,,,,,,
1990,"52,233","133,370","178,444","204,442","222,193","232,940","253,337","256,788","261,166","263,000",,,,,,,,,
1991,"59,315","128,051","169,793","196,685","213,165","234,676","239,195","245,499","247,131","248,319",,,,,,,,,
1992,"39,991","89,873","114,117","133,003","154,362","159,496","164,013","166,212","167,397","168,844",,,,,,,,,
1993,"19,744","47,229","61,909","85,099","87,215","88,602","89,444","89,899","90,446","90,686",,,,,,,,,
1994,"20,379","46,773","88,636","91,077","92,583","93,346","93,897","94,165","94,558","94,730",,,,,,,,,
1995,"18,756","84,712","87,311","89,200","90,001","90,247","90,687","91,068","91,001","91,161",,,,,,,,,
1996,"42,609","44,916","46,981","47,899","48,583","49,109","49,442","49,073","49,161","49,255",,,,,,,,,
1997,691,"2,085","2,795","2,866","2,905","2,909","2,908","2,909","2,909","2,909",,,,,,,,,


In [33]:
triangle["CaseIncurLoss"] = triangle["IncurLoss_D"] - triangle["BulkLoss_D"]
triangle["PaidToInc"] = triangle["CumPaidLoss_D"] / triangle["IncurLoss_D"]
triangle[["CaseIncurLoss", "PaidToInc"]]

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 2, 19, 19)"
Index:,[GRNAME]
Columns:,"[CaseIncurLoss, PaidToInc]"


In [34]:
triangle["CumPaidLoss_D"].sum() / triangle["EarnedPremNet_D"].sum()

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,0.1690,0.3776,0.5116,0.5892,0.6412,0.6701,0.6917,0.7078,0.7269,0.7343,,,,,,,,,
1989,0.1712,0.3805,0.5100,0.5927,0.6419,0.6733,0.6952,0.7183,0.7279,0.7368,,,,,,,,,
1990,0.1703,0.4028,0.5409,0.6217,0.6696,0.6995,0.7277,0.7417,0.7538,0.7613,,,,,,,,,
1991,0.1683,0.3844,0.5102,0.5858,0.6331,0.6691,0.6852,0.7008,0.7097,0.7164,,,,,,,,,
1992,0.1567,0.3569,0.4636,0.5357,0.5855,0.6069,0.6262,0.6394,0.6485,0.6567,,,,,,,,,
1993,0.1379,0.2995,0.3863,0.4484,0.4784,0.5010,0.5148,0.5244,0.5325,0.5386,,,,,,,,,
1994,0.1353,0.2892,0.3830,0.4296,0.4601,0.4791,0.4918,0.5020,0.5094,0.5168,,,,,,,,,
1995,0.1314,0.2937,0.3677,0.4157,0.4449,0.4625,0.4754,0.4834,0.4902,0.4970,,,,,,,,,
1996,0.1576,0.3041,0.3875,0.4366,0.4673,0.4886,0.5035,0.5122,0.5206,0.5264,,,,,,,,,
1997,0.1541,0.3310,0.4261,0.4831,0.5174,0.5398,0.5540,0.5657,0.5720,0.5786,,,,,,,,,


In [35]:
triangle.groupby("GRNAME").sum()

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(132, 9, 19, 19)"
Index:,[GRNAME]
Columns:,"[DevelopmentLag, IncurLoss_D, CumPaidLoss_D, BulkLoss_D, EarnedPremDIR_D, EarnedPremCeded_D, EarnedPremNet_D, CaseIncurLoss, PaidToInc]"


In [39]:
pd.DataFrame(np.unique(triangle["GRNAME"]))

,0
0,Agway Ins Co
1,Alaska Nat Ins Co
2,Alaska Timber Ins Exchange
3,Allstate Ins Co Grp
4,American Contractors Ins Grp
...,...
127,Wisconsin American Mut Ins Co
128,Wisconsin Cnty Mut Ins Corp
129,Workers Comp Exch
130,Yasuda Fire & Marine Ins Co Of Amer


In [37]:
triangle.sum(axis="index").sum(axis="segments")

,Triangle Summary
Valuation:,2006-12
Grain:,OYDY
Shape:,"(1, 9, 19, 19)"
Index:,[GRNAME]
Columns:,"[DevelopmentLag, IncurLoss_D, CumPaidLoss_D, BulkLoss_D, EarnedPremDIR_D, EarnedPremCeded_D, EarnedPremNet_D, CaseIncurLoss, PaidToInc]"


In [38]:
triangle.sum(axis=0).sum(axis=1)

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,"6,505,334","6,998,133","7,251,426","7,372,627","7,516,601","7,562,091","7,569,563","7,595,274","7,627,302","7,628,929",,,,,,,,,
1989,"6,993,589","7,485,209","7,703,016","7,929,058","8,019,964","8,051,545","8,085,541","8,122,084","8,143,175","8,135,914",,,,,,,,,
1990,"7,497,213","8,222,881","8,450,718","8,597,213","8,651,657","8,678,904","8,729,039","8,765,898","8,777,143","8,763,675",,,,,,,,,
1991,"8,307,773","8,883,966","9,129,062","9,234,930","9,289,198","9,361,264","9,399,476","9,425,834","9,415,632","9,407,692",,,,,,,,,
1992,"8,511,972","9,181,979","9,294,409","9,316,729","9,384,021","9,438,707","9,473,071","9,476,289","9,494,934","9,489,522",,,,,,,,,
1993,"9,280,994","9,671,786","9,746,181","9,725,504","9,784,804","9,818,267","9,810,062","9,782,090","9,781,425","9,771,582",,,,,,,,,
1994,"9,631,728","9,962,769","9,981,869","9,963,848","9,984,381","9,994,971","9,983,078","9,979,242","9,985,671","10,000,530",,,,,,,,,
1995,"9,607,355","9,916,423","9,953,288","9,866,027","9,890,350","9,871,873","9,881,837","9,885,418","9,915,027","9,945,121",,,,,,,,,
1996,"8,998,679","9,221,765","9,366,872","9,354,414","9,347,331","9,366,417","9,382,955","9,404,258","9,439,836","9,467,252",,,,,,,,,
1997,"8,271,236","8,668,566","8,855,925","8,865,622","8,899,831","8,956,443","8,984,406","9,026,011","9,066,499","9,083,781",,,,,,,,,
